In [1]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("CarPrice_Assignment.csv")

# Drop unnecessary columns
df.drop(['car_ID', 'CarName'], axis=1, inplace=True)

# Convert categorical variables
df = pd.get_dummies(df, drop_first=True)

# Split features & target
X = df.drop('price', axis=1)
y = df['price']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scaling (important for SVR & Linear Regression)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR()
}


In [3]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        "Model": name,
        "R2": r2_score(y_test, y_pred),
        "MSE": mean_squared_error(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred)
    })

results_df = pd.DataFrame(results)
results_df


,Model,R2,MSE,MAE
0,Linear Regression,0.892557,8.482008e+06,2089.382729
1,Decision Tree,0.894859,8.300272e+06,1886.321146
2,Random Forest,0.958021,3.313970e+06,1259.757695
3,Gradient Boosting,0.925262,5.900139e+06,1676.027625
4,SVR,-0.099891,8.682982e+07,5697.657697


In [4]:
importances = models["Gradient Boosting"].feature_importances_

feature_imp = pd.Series(importances, index=X.columns)
feature_imp.sort_values(ascending=False).head(10)


enginesize             0.596489
curbweight             0.163463
highwaympg             0.070945
horsepower             0.040743
cylindernumber_four    0.038829
wheelbase              0.016322
carlength              0.014109
carwidth               0.013019
boreratio              0.008593
stroke                 0.007426
dtype: float64

In [5]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 4]
}

gbr = GradientBoostingRegressor(random_state=42)

grid = GridSearchCV(gbr, params, cv=5, scoring='r2')
grid.fit(X_train, y_train)

best_model = grid.best_estimator_


In [6]:
y_pred_best = best_model.predict(X_test)

print("Improved R2:", r2_score(y_test, y_pred_best))


Improved R2: 0.9282655992830067
